# SmartBeds - One-Class entrenado con crisis

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [4]:
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import pickle as pk
from sklearn import metrics as mtr
from sklearn import svm

In [33]:
nombre = "data/crisis_18-nov-10-"
datos = {'Raw':nombre+"raw.pdd",'SavGol':nombre+"savgol.pdd",'Butter':nombre+"butter.pdd",
         'Stats':nombre+"stats.pdd",'SavGol-Stats':nombre+"savgol-stats.pdd","Butter-Stats":nombre+"butter-stats.pdd"}

### Trainer

In [6]:
def train(dataframe):
    data = dataframe.iloc[:,1:len(dataframe)-1]
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(data)
    return clf
def predict(dataframe,classifier):
    data = dataframe.iloc[:,1:len(dataframe)-1]
    return classifier.predict(data)

### Raw

In [39]:
with open(datos['Raw'],'rb') as f:
    raw = pk.load(f)
crisis = raw[raw['target']==True]
no_crisis = raw[raw['target']==False]
trn = no_crisis.iloc[:,1:len(no_crisis.columns)-1]
trn.head()

,P1,P2,P3,P4,P5,P6
3849685,4.098361,18.852459,22.131148,22.131148,13.114754,6.557377
3849686,4.098361,18.852459,21.311475,22.131148,13.114754,6.557377
3849687,4.918033,19.672131,22.131148,22.950820,13.934426,7.377049
3849688,4.098361,18.852459,21.311475,22.131148,13.114754,6.557377
3849689,0.000000,18.852459,22.131148,22.950820,13.114754,6.557377


In [40]:
raw_clf = train(trn)

KeyboardInterrupt: 

In [9]:
y = predict(raw,raw_clf)
y[y==-1]=0
y=y.astype('bool')

In [10]:
mtr.confusion_matrix(raw['target'],y)

array([[10522, 93824],
       [  612,    58]])

### SavGol

In [11]:
with open(datos['SavGol'],'rb') as f:
    savgol = pk.load(f)
crisis = savgol[savgol['target']==True]
no_crisis = savgol[savgol['target']==False]

In [12]:
savgol_clf = train(no_crisis)

In [13]:
y = predict(savgol,savgol_clf)
y[y==-1]=0
y=y.astype('bool')

In [14]:
mtr.confusion_matrix(savgol['target'],y)

array([[10432, 93914],
       [  670,     0]])

### Butter

In [15]:
with open(datos['Butter'],'rb') as f:
    butter = pk.load(f)
crisis = butter[butter['target']==True]
no_crisis = butter[butter['target']==False]

In [16]:
butter_clf = train(no_crisis)

In [17]:
y = predict(butter,butter_clf)
y[y==-1]=0
y=y.astype('bool')

In [18]:
mtr.confusion_matrix(butter['target'],y)

array([[10439, 93907],
       [  670,     0]])

### Stats

In [19]:
with open(datos['Stats'],'rb') as f:
    stats = pk.load(f).dropna()
crisis = stats[stats['target']==True]
no_crisis = stats[stats['target']==False]

In [20]:
stats_clf = train(no_crisis)

In [21]:
y = predict(stats,stats_clf)
y[y==-1]=0
y=y.astype('bool')

In [22]:
mtr.confusion_matrix(stats['target'],y)

array([[10882, 93440],
       [  670,     0]])

### SavGol Stats

In [23]:
with open(datos['SavGol-Stats'],'rb') as f:
    savgol_s = pk.load(f).dropna()
crisis = savgol_s[savgol_s['target']==True]
no_crisis = savgol_s[savgol_s['target']==False]

In [24]:
savgol_s_clf = train(no_crisis)

In [25]:
y = predict(savgol_s,savgol_s_clf)
y[y==-1]=0
y=y.astype('bool')

In [26]:
mtr.confusion_matrix(savgol_s['target'],y)

array([[10445, 93877],
       [  670,     0]])

### Butter Stats

In [27]:
with open(datos['Butter-Stats'],'rb') as f:
    butter_s = pk.load(f).dropna()
crisis = butter_s[butter_s['target']==True]
no_crisis = butter_s[butter_s['target']==False]

In [28]:
butter_s_clf = train(no_crisis)

In [29]:
y = predict(butter_s,butter_s_clf)
y[y==-1]=0
y=y.astype('bool')

In [30]:
mtr.confusion_matrix(butter_s['target'],y)

array([[10451, 93871],
       [  670,     0]])

### Almacenado de clasificadores

In [31]:
clasificadores = [raw_clf,savgol_clf,butter_clf,stats_clf,savgol_s_clf,butter_s_clf]
nombres = ["RAW","SAVGOL","BUTTER","STATS","SAVGOL-STATS","BUTTER_STATS"]
for i in range(len(nombres)):
    with open("classifiers/"+nombres[i]+"-no_seizure-classifier.pkl",'wb') as f:
        pk.dump(clasificadores[i],f)